<a href="https://colab.research.google.com/github/Jacob-Rose-BU/Alternative-Investments---Assette-Capstone-Project/blob/main/PORTFOLIOPERFORMANCE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##Portfolio Peformance Table##

**Columns Modified From Assette:**

PORTFOLIOCODE: Portfolio identifier

HISTORYDATE: Date of performance record

CURRENCYCODE: Currency code (e.g., USD)

CURRENCY: Human-readable currency name

PERFORMANCECATEGORY: Broad classification (e.g., “Total Return”)

PERFORMANCECATEGORYNAME: More descriptive category name

PERFORMANCETYPE: Always “Holdings-Based”

PERFORMANCEPERIOD: Period label (e.g., YTD, 1M, 3M, 1Y)

PERFORMANCEFREQUENCY: Frequency of return calculation (e.g., Daily)

PERFORMANCEFACTOR: The calculated return value

**Columns Created from Yfinance:**

BENCHMARKCODE: Identifier for the benchmark index (e.g., SP500ESG)

ALPHA: Holdings-based alpha (portfolio return – benchmark return)


In [61]:
import yfinance as yf
import pandas as pd
import random
from datetime import datetime

NUM_PORTFOLIOS = 5
SECURITIES_PER_PORTFOLIO = 5
TODAY = datetime.today()
TODAY_STR = TODAY.strftime('%Y-%m-%d')
YEAR_START = f"{TODAY.year}-01-01"
esg_tickers = ['AAPL', 'MSFT', 'ADBE', 'NVDA', 'TSLA', 'GOOGL', 'AMZN', 'INTC', 'META', 'V']

In [62]:
# Benchmark setup
benchmark_ticker = "^SPESG"
benchmark_data = yf.download(benchmark_ticker, period="2d", interval="1d", progress=False)

# Use 'Close' instead of 'Adj Close'
if 'Close' in benchmark_data.columns:
    benchmark_prices = benchmark_data['Close']
else:
    raise ValueError("Benchmark 'Close' price not found!")

benchmark_return = round(((benchmark_prices.iloc[-1] - benchmark_prices.iloc[-2]) / benchmark_prices.iloc[-2]), 4)
benchmark_code = "SP500ESG"

/tmp/ipython-input-836885512.py:3: FutureWarning: YF.download() has changed argument auto_adjust default to True
  benchmark_data = yf.download(benchmark_ticker, period="2d", interval="1d", progress=False)


In [63]:
# Define performance periods and their start dates
performance_periods = {
    "YTD": f"{TODAY.year}-01-01",
    "1M": (TODAY - pd.DateOffset(days=30)).strftime('%Y-%m-%d'),
    "3M": (TODAY - pd.DateOffset(days=90)).strftime('%Y-%m-%d'),
    "1Y": (TODAY - pd.DateOffset(days=365)).strftime('%Y-%m-%d')
}

In [64]:
# Portfolio generation
portfolio_data = []
for i in range(NUM_PORTFOLIOS):
    tickers = random.sample(esg_tickers, SECURITIES_PER_PORTFOLIO)
    weights = [random.uniform(0.1, 0.4) for _ in range(SECURITIES_PER_PORTFOLIO)]
    total = sum(weights)
    normalized_weights = [round(w / total, 4) for w in weights]
    portfolio_data.append((f"PF{i+1:03}", tickers, normalized_weights))

In [72]:
# Calculate performance
rows = []

for portfolio_code, tickers, weights in portfolio_data:
    try:
        # Get 2-day data for alpha calculation
        price_data = yf.download(tickers, period="2d", interval="1d", progress=False)
        if isinstance(price_data.columns, pd.MultiIndex):
            price_data = price_data.xs("Close", axis=1, level=0)
        else:
            price_data = price_data[["Close"]]
            price_data.columns = [tickers[0]]

        prev_prices = price_data.iloc[0]
        curr_prices = price_data.iloc[1]
        security_returns = (curr_prices - prev_prices) / prev_prices
        account_return = float(sum(w * r for w, r in zip(weights, security_returns)))
        alpha = round(account_return - float(benchmark_return), 4)

        # Loop through all periods and calculate each return
        for period_label, start_date in performance_periods.items():
            period_data = yf.download(tickers, start=start_date, end=TODAY_STR, progress=False)
            if isinstance(period_data.columns, pd.MultiIndex):
                period_data = period_data.xs("Close", axis=1, level=0)
            else:
                period_data = period_data[["Close"]]
                period_data.columns = [tickers[0]]

            # Skip if not enough data
            if period_data.shape[0] < 2:
                continue

            period_returns = (period_data.iloc[-1] - period_data.iloc[0]) / period_data.iloc[0]
            portfolio_return = float(sum(w * r for w, r in zip(weights, period_returns)))

            rows.append({
                "PORTFOLIOCODE": portfolio_code,
                "HISTORYDATE": TODAY_STR,
                "CURRENCYCODE": "USD",
                "CURRENCY": "USD",
                "PERFORMANCECATEGORY": "Total Return",
                "PERFORMANCECATEGORYNAME": "Holdings-Based",
                "PERFORMANCETYPE": "Holdings-Based",
                "PERFORMANCEPERIOD": period_label,
                "PERFORMANCEINCEPTIONDATE": None,
                "PORTFOLIOINCEPTIONDATE": None,
                "PERFORMANCEFREQUENCY": "Daily",
                "PERFORMANCEFACTOR": round(portfolio_return, 4),
                "BENCHMARKCODE": benchmark_code,
                "ALPHA": alpha if period_label == "YTD" else None
            })

    except Exception as e:
        print(f"❌ Error calculating portfolio {portfolio_code}: {e}")
        continue

/tmp/ipython-input-1649002961.py:7: FutureWarning: YF.download() has changed argument auto_adjust default to True
  price_data = yf.download(tickers, period="2d", interval="1d", progress=False)
/tmp/ipython-input-1649002961.py:18: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  alpha = round(account_return - float(benchmark_return), 4)
/tmp/ipython-input-1649002961.py:22: FutureWarning: YF.download() has changed argument auto_adjust default to True
  period_data = yf.download(tickers, start=start_date, end=TODAY_STR, progress=False)
/tmp/ipython-input-1649002961.py:22: FutureWarning: YF.download() has changed argument auto_adjust default to True
  period_data = yf.download(tickers, start=start_date, end=TODAY_STR, progress=False)
/tmp/ipython-input-1649002961.py:22: FutureWarning: YF.download() has changed argument auto_adjust default to True
  period_data = yf.download(tickers, start=start

In [73]:
# Output
df = pd.DataFrame(rows)
timestamp = datetime.utcnow().strftime('%Y%m%d_%H%M%S')
filename = f"portfolio_performance_upload_{timestamp}.csv"
df.to_csv(filename, index=False)

print(f"✅ Exported portfolio performance data to: {filename}")
df

✅ Exported portfolio performance data to: portfolio_performance_upload_20250806_190055.csv


,PORTFOLIOCODE,HISTORYDATE,CURRENCYCODE,CURRENCY,PERFORMANCECATEGORY,PERFORMANCECATEGORYNAME,PERFORMANCETYPE,PERFORMANCEPERIOD,PERFORMANCEINCEPTIONDATE,PORTFOLIOINCEPTIONDATE,PERFORMANCEFREQUENCY,PERFORMANCEFACTOR,BENCHMARKCODE,ALPHA
0,PF001,2025-08-06,USD,USD,Total Return,Total Return (Holdings-Based),Holdings-Based,YTD,None,None,Daily,-0.0702,SP500ESG,0.0134
1,PF001,2025-08-06,USD,USD,Total Return,Total Return (Holdings-Based),Holdings-Based,1M,None,None,Daily,0.0187,SP500ESG,NaN
2,PF001,2025-08-06,USD,USD,Total Return,Total Return (Holdings-Based),Holdings-Based,3M,None,None,Daily,0.1268,SP500ESG,NaN
3,PF001,2025-08-06,USD,USD,Total Return,Total Return (Holdings-Based),Holdings-Based,1Y,None,None,Daily,0.2024,SP500ESG,NaN
4,PF002,2025-08-06,USD,USD,Total Return,Total Return (Holdings-Based),Holdings-Based,YTD,None,None,Daily,0.0671,SP500ESG,0.0119
5,PF002,2025-08-06,USD,USD,Total Return,Total Return (Holdings-Based),Holdings-Based,1M,None,None,Daily,0.0057,SP500ESG,NaN
6,PF002,2025-08-06,USD,USD,Total Return,Total Return (Holdings-Based),Holdings-Based,3M,None,None,Daily,0.1393,SP500ESG,NaN
7,PF002,2025-08-06,USD,USD,Total Return,Total Return (Holdings-Based),Holdings-Based,1Y,None,None,Daily,0.3602,SP500ESG,NaN
8,PF003,2025-08-06,USD,USD,Total Return,Total Return (Holdings-Based),Holdings-Based,YTD,None,None,Daily,0.0416,SP500ESG,0.0020
9,PF003,2025-08-06,USD,USD,Total Return,Total Return (Holdings-Based),Holdings-Based,1M,None,None,Daily,0.0006,SP500ESG,NaN
